In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys 
sys.path.append("/Users/julianganzabal/facultad/lab-ml/mllab-tools")

In [3]:
from RNN_utils import get_deep_rnn, chars_to_one_hot, sample
import numpy as np

Using TensorFlow backend.


In [4]:
model=get_deep_rnn((100, 69), dense_units=69)

input shape =  (100, 69)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 200)          216000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dense_1 (Dense)              (None, 69)                13869     
Total params: 550,669
Trainable params: 550,669
Non-trainable params: 0
_________________________________________________________________


# Cargo pesos entrenados con 100 epochs

In [5]:
# bodes_sangre_LSTM_deep
# bodes_sangre_LSTM_deep_dropout_04
model.load_weights('bodes_sangre_LSTM_deep_dropout_04.hdf5')

In [6]:
def split_data_and_get_dicts(text, train_fraction=0.6):
    number_of_chars = len(text)
    validation_index = int(number_of_chars*train_fraction)
    text_train = text[:validation_index]
    text_validation = text[validation_index:]
    chars_train = set(text_train)
    chars_test = set(text_validation)
    chars_set = chars_train.intersection(chars_test)
    chars = sorted(list(chars_set))
    chars_to_indices = dict((c, i) for i, c in enumerate(chars))  # map each unique character to unique integer
    indices_to_chars = dict((i, c) for i, c in enumerate(chars))  # map each unique integer back to unique character
    return text_train, text_validation, chars_to_indices, indices_to_chars, chars

In [7]:
text = open('bodas_de_sangre.txt').read()
text_train, text_validation, chars_to_indices, indices_to_chars, chars = split_data_and_get_dicts(text)

In [8]:
window_size = 100
initial_char = 16
initial_text = text_validation[initial_char:window_size+initial_char]
print(initial_text)

MOZO 1: (Entrando) ¡Tienes que beber con nosotros!

NOVIO: Estoy esperando a la novia.

MOZO 2: ¡Ya 


In [9]:
X_test = chars_to_one_hot(initial_text, chars, chars_to_indices, window_size)
print(X_test.shape)

(1, 100, 69)


In [10]:
probs = model.predict(X_test)
print(probs)
index = np.argmax(probs)
print(index)
print(indices_to_chars[index])

[[  2.67503987e-04   1.47814702e-04   6.57919809e-05   2.55769206e-04
    4.72744568e-06   2.37566219e-05   2.42656406e-05   2.75124034e-06
    3.70589163e-07   4.40745573e-08   3.66884683e-06   1.64531014e-06
    6.16733669e-05   7.41606846e-06   1.38893829e-05   8.09140984e-05
    5.61719717e-05   4.66867023e-05   1.43318921e-05   3.72986892e-06
    7.39483858e-06   2.96074359e-05   2.25158919e-06   1.05969375e-03
    2.02942902e-04   3.09547031e-05   3.36788726e-05   1.23046717e-04
    3.96312171e-05   1.16490744e-04   6.03278386e-05   4.03930717e-05
    1.16660331e-05   1.97234622e-05   2.18111982e-05   1.14413998e-08
    3.21773347e-03   1.22510111e-02   5.05333841e-02   1.11963928e-01
    7.49744847e-02   1.27892541e-02   1.42459823e-02   2.28233393e-02
    8.94578919e-03   8.82902532e-04   1.80775281e-02   6.82415739e-02
    1.20949827e-01   1.26354326e-03   6.70492798e-02   4.00201790e-02
    8.58152192e-03   8.35077539e-02   9.80713218e-02   1.13407187e-02
    1.37485445e-01  

## Muestrar modelo

In [11]:
temperatura = 1
sample([0.5, 0.25, 0.15, 0.10], temperatura, verbose=True)

[ 0.5   0.25  0.15  0.1 ]


0

# Ejemplos con 100 epochs
### Primer muestreo, luego del espacio despues de la palabra 'ya' 

In [12]:
window_size = 100
initial_char = 16
initial_text = text_validation[initial_char:window_size+initial_char]
print(initial_text)
X_test = chars_to_one_hot(initial_text, chars, chars_to_indices, window_size)
probs = model.predict(X_test)
for i in range(10):
    my_sample = sample(probs[0], 1)
    print(indices_to_chars[my_sample])

MOZO 1: (Entrando) ¡Tienes que beber con nosotros!

NOVIO: Estoy esperando a la novia.

MOZO 2: ¡Ya 
q
d
e
m
p
t
p
f
c
n


### Luego de las letras 'novi'

In [13]:
window_size = 100
initial_char = 0
initial_text = text_validation[initial_char:window_size+initial_char]
print(initial_text)
X_test = chars_to_one_hot(initial_text, chars, chars_to_indices, window_size)
probs = model.predict(X_test)
for i in range(10):
    my_sample = sample(probs[0], 1)
    print(indices_to_chars[my_sample])

ovia. (Se va.)

MOZO 1: (Entrando) ¡Tienes que beber con nosotros!

NOVIO: Estoy esperando a la novi
a
a
a
a
a
a
a
a
a
a


In [14]:
# funcion de generación de texto
def generate_text(model, initial_text, N = 1000, window_size = 100):
    X_text_str = initial_text
    print(initial_text)
    print('--------------------------------------------------------------------------------')
    print()
    print(X_text_str, end='')
    for i in range(N):
        X_test = chars_to_one_hot(X_text_str[i:], chars, chars_to_indices, window_size)
        probs = model.predict(X_test)
        my_sample=sample(probs[0], 0.5)
        new_char = indices_to_chars[my_sample]
        X_text_str = X_text_str + new_char
        print(new_char, end='')
    return X_text_str

# Poco entrenamiento: 1 Epoch

In [18]:
model.load_weights('bodes_sangre_LSTM_deep_dropout_04_2_1_epochs.hdf5')
initial_char = 16
window_size = 100
initial_text = text_validation[initial_char:window_size+initial_char]
model.load_weights('bodes_sangre_LSTM_deep_dropout_04_2_1_epochs.hdf5')
generate_text(model, initial_text, N = 1000, window_size = window_size)

MOZO 1: (Entrando) ¡Tienes que beber con nosotros!

NOVIO: Estoy esperando a la novia.

MOZO 2: ¡Ya 
--------------------------------------------------------------------------------

MOZO 1: (Entrando) ¡Tienes que beber con nosotros!

NOVIO: Estoy esperando a la novia.

MOZO 2: ¡Ya ae ar o te e enoe a co pe en lorde an os aata orre e aa de e a aa qc e tl lo ce an as le ea a e e o ca a mn ta a iu a pa den. ao ma anne ca a e ena lo ar a ena era da e eo oa a e ee men.

MOEIR:  ae ae oras a le mon.


VAA : ¿io mo no te la men da ae ao a naa ae mua arga ere nu e au a cla io to aa a de anta e ar. tes uone o eue are ae en e a no do la ade are do ve mo eera a ara a nu a o re a e cutr nre o ue nr e oe ys ta ea a le aiv lo era o a anera ea o ae ana as an ne as e o te que as en e ara sa end.

UNIOA: :sio a e ora ada.

NIAI: ¿o esrvrr on ao lo  ua eula o u oe e mn na ae e ace e pl os o ne me de e a ara o ca mo e a anla are es esrs ae ta. do  o era aa ve ad ono.

NNIND:: Soran are )a nio al qu a o 

# Corrida 1 (100 epochs)

In [16]:
model.load_weights('bodes_sangre_LSTM_deep_dropout_04.hdf5')
output_text = generate_text(model, initial_text, N = 1000, window_size = window_size)

ovia. (Se va.)

MOZO 1: (Entrando) ¡Tienes que beber con nosotros!

NOVIO: Estoy esperando a la novi
--------------------------------------------------------------------------------

ovia. (Se va.)

MOZO 1: (Entrando) ¡Tienes que beber con nosotros!

NOVIO: Estoy esperando a la novia. De puede una mujer de mu hijo. (La mujer el azahar.) ¿Qué te alguna con el azahar?

MADRE: Sí. ¿Qué me hace la para tento?

NOVIO: Yo sombrera.

MADRE: ¿Qué están yo está que me siento que salí?

LEONARDO: No. Yo se cuerta la mana verdad. (Se sienta.)

VECINA: (Al criada) La madre de aballo.

MADRE: (Seriendo) Sombre a tu casa.

MADRE: Eso me mujer.

MADRE: Yo quiero está de la sonque hay a la caballo. 

CRIADA: Pero la cero un novio.

NOVIO: ¡Como los novio!

CRIADA: Estan muser.

NOVIA: Pero con el caballo y a pare a la cara.

LEONARDO: Vamos.

CRIADA: Esto es brazos y mujer.

NOVIA: Se.

MADRE: Tu padre que priso por el ascho y la carrende. Pero no me hombre hablar con la guente.

CRIADA: (Levantándose

### Corrida 2

In [17]:
output_text = generate_text(model, initial_text, N = 1000, window_size = window_size)

ovia. (Se va.)

MOZO 1: (Entrando) ¡Tienes que beber con nosotros!

NOVIO: Estoy esperando a la novi
--------------------------------------------------------------------------------

ovia. (Se va.)

MOZO 1: (Entrando) ¡Tienes que beber con nosotros!

NOVIO: Estoy esperando a la novia. (Se esa carallo de la mujer) (Se diente y a la caballo que no sabes a la marada de azahar) No sé de pasa. (La madre con el cabello y lo viviera a la carecia) No sé como siente la casa matar de una mujer. Por el no te pueden a mí le vista. (Se sienta en tu pecho en el caballo.) 

MUJER: (Al niña) ¡Qué está como está para la caballe de la cara a la cabella y con las campos y a casardo en la boda.

NOVIA: (Meriendo) Qué con el caballo y con el caballo y se casa casale de arredor.

MADRE: No se cuere por el cabello. (Se sienta a la cara y puede a la elca de la colona! Mana con las destido en un eltente. Está bien a la novia en una comprando de la para de la boda. (Se sienta al niño a me puede a manida.)

MADR

- Mantiene estructuta de obra de teatro
- Abre y cierra signo de pregunta y/o admiración
- Abre y cierra parentesis
- Articulo mas sustantivo con género
- Comparar primera y segunda corrida y verificar que son diferentes